In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms

import matplotlib

import matplotlib.pyplot as plt

import os
import random
import math

from torch.utils.data.dataset import Dataset

INPUT_PATH = "./output/nodule_npy/malignancy/"

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 20
nz = 100
n_epochs = 100
ngf = 64
ngpu = 1
print(device)

cuda:0


In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def chunks(arr, m):
    nchunk = int(math.ceil(len(arr) / float(m)))
    return [arr[i:i + nchunk] for i in range(0, len(arr), nchunk)]

def five_folder(arr, number):
    training_set = []
    test_set = []
    for j in range(len(arr)):
        if number == j:
            test_set.extend(arr[j])
        else:
            training_set.extend(arr[j])
    return training_set, test_set


class MyDataset(Dataset):
    def __init__(self, images):
        self.images = images

    def __getitem__(self, index):#返回的是tensor
        img = self.images[index]
        return img

    def __len__(self):
        return len(self.images)

In [4]:
# Generator
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu

        self.project = nn.Sequential(
          nn.Linear(nz, 1 * 5 * 5 * ngf * 4, bias=False)
        )
        self.deconv = nn.Sequential(
          # input is Z, going into a deconvolution
          # state size. BATCH_SIZE x (ngf*8) x 4 x 4 x 4
#           nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size=4, stride=2, padding=1, bias=False),
#           nn.BatchNorm3d(ngf * 4),
#           nn.ReLU(True),
          # state size. BATCH_SIZE x (ngf*4) x 1 x 5 x 5
          nn.ConvTranspose3d(ngf * 4, ngf * 2, kernel_size=4, stride=2, padding=1, bias=False),
          nn.BatchNorm3d(ngf * 2),
          nn.ReLU(True),
          # state size. BATCH_SIZE x (ngf*2) x 2 x 10 x 10
          nn.ConvTranspose3d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1, bias=False),
          nn.BatchNorm3d(ngf),
          nn.ReLU(True),
          # state size. BATCH_SIZE x (ngf) x 4 x 20 x 20
          nn.ConvTranspose3d(ngf, 1, kernel_size=4, stride=2, padding=1, bias=False),
          nn.Tanh()
          # state size. BATCH_SIZE x 1 x 8 x 40 x 40
        )

    def forward(self, input):
        x = self.project(input)
        # Conv3d的规定输入数据格式为(batch, channel, Depth, Height, Width)
#         print(x.shape)
        x = x.view(-1, ngf * 4, 1, 5, 5)
#         x = self.deconv(x)
        
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.deconv, x, range(self.ngpu))
        else:
            output = self.deconv(x)

        return output


# Discriminator
def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)), dim=-1)
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=(2,9,9)):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv3d(in_channels=in_channels,
                                out_channels=out_channels,
                                kernel_size=kernel_size,
                                stride=1
                              )

    def forward(self, x):
        return F.relu(self.conv(x))


class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=(2,9,9), num_routes=32 * 12 * 12 * 6):
        super(PrimaryCaps, self).__init__()

        self.capsules = nn.ModuleList([
          nn.Conv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=(1,2,2), padding=0) 
                      for _ in range(num_capsules)])
  
    def forward(self, x):
        u = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
        u = torch.cat(u, dim=-1)
        return self.squash(u)
  
    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor



class DigitCaps(nn.Module):
    def __init__(self, num_capsules=1, num_routes=32 * 12 * 12 * 6, in_channels=8, out_channels=16, num_iterations=3):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules
        self.num_iterations = num_iterations
        self.route_weights = nn.Parameter(torch.randn(num_capsules, num_routes, in_channels, out_channels)).to(device)

    def forward(self, x):
        # 矩阵相乘
        # x.size(): [1, batch_size, in_capsules, 1, dim_in_capsule]
        # weight.size(): [num_capsules, 1, num_route, in_channels, out_channels]
        priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]
        logits = Variable(torch.zeros(*priors.size())).to(device)

        for i in range(self.num_iterations):
            probs = softmax(logits, dim=2)
            outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))

            if i != self.num_routes - 1:
                delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
                logits = logits + delta_logits
        
        return outputs.squeeze()
  
    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
    
        self.reconstraction_layers = nn.Sequential(
          nn.Linear(16 * 10, 512),
          nn.ReLU(inplace=True),
          nn.Linear(512, 1024),
          nn.ReLU(inplace=True),
          nn.Linear(1024, 784),
          nn.Sigmoid()
        )
      
    def forward(self, x, data):
        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes, dim=-1)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(10))
        if USE_CUDA:
              masked = masked.cuda()
        masked = masked.index_select(dim=0, index=max_length_indices.squeeze(1).data)
    
        reconstructions = self.reconstraction_layers((x * masked[:, :, None, None]).view(x.size(0), -1))
        reconstructions = reconstructions.view(-1, 1, 28, 28)

        return reconstructions, masked


class CapsNet(nn.Module):
    def __init__(self, ngpu):
        super(CapsNet, self).__init__()
        self.ngpu = ngpu
        
        self.conv_layer = ConvLayer()
        self.primary_capsules = PrimaryCaps()
        self.digit_capsules = DigitCaps()

        # self.decoder = Decoder()
        # self.mse_loss = nn.MSELoss()
      
    def forward(self, data):
        if data.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.conv_layer, data, range(self.ngpu))
            output = nn.parallel.data_parallel(self.primary_capsules, output, range(self.ngpu))
            output = nn.parallel.data_parallel(self.digit_capsules, output, range(self.ngpu))
        else:
            output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))

        classes = (output ** 2).sum(dim=-1) ** 0.5        
        return classes, output

        #reconstructions, masked = self.decoder(output, data)
        #return output, reconstructions, masked

    # def loss(self, data, x, target, reconstructions):
      #   return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    # def margin_loss(self, x, labels, size_average=True):
      #   batch_size = x.size(0)

      #   v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))

      #   left = F.relu(0.9 - v_c).view(batch_size, -1)
      #   right = F.relu(v_c - 0.1).view(batch_size, -1)

      #   loss = labels * left + 0.5 * (1.0 - labels) * right
      #   loss = loss.sum(dim=1).mean()

      #   return loss

    # def reconstruction_loss(self, data, reconstructions):
      #   loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
      #   return loss * 0.0005


class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()

    def forward(self, classes, labels):
        left = F.relu(0.9 - classes, inplace=True) ** 2
        right = F.relu(classes - 0.1, inplace=True) ** 2

        margin_loss = labels * left + 0.5 * (1. - labels) * right
        margin_loss = margin_loss.sum()

        return margin_loss / classes.size(0)

In [5]:
netD_list = []
netG_list = []
D_losses_list = []
G_losses_list = []

def train(train_loader, single_test_set):
    netD = CapsNet(ngpu).to(device)
    criterion = CapsuleLoss()

    netG = Generator(ngpu).to(device)
    netG.apply(weights_init)

    fixed_noise = torch.randn(BATCH_SIZE, nz, device=device)
    real_label = 1
    fake_label = 0

    optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    D_losses = []
    G_losses = []

    for epoch in range(n_epochs):
        print("-----THE [{}/{}] epoch start-----".format(epoch + 1, n_epochs))
        
        for j, data in enumerate(train_loader, 0):
#                     print(j, data.shape) #torch.Size([32, 1, 8, 40, 40])

#                     plt.figure()
#                     for lenc in range(data.shape[0]):
#                         for len_img in range(8):
#                             plt.subplot(2, 4, len_img + 1)
#                             pixel_array = data[lenc][0][len_img]
#                             print(pixel_array)
#                             plt.imshow(pixel_array, cmap="gray")
#                         plt.show()


#                     pixel_array = data[0][0][0][20]
#                     print(pixel_array)

#                     data = data / 255.0
#                     pixel_array = data[0][0][0][20]
#                     print(pixel_array)
            ############################
            # (1) Update D network: minimize Lm(D(x), T = 1) + Lm(D(G(z)), T = 0)
            ###########################
            # train with real
            
            netD.train()
            
            netD.zero_grad()
            real_cpu = data.to(device, dtype=torch.float)
            batch_size = real_cpu.size(0)
            label = torch.full((batch_size,), real_label, device=device)   

            classes_d_real, output_d_real = netD(real_cpu)
#                     print("ERR_D_REAL:",classes_d_real)
            errD_real = criterion(classes_d_real, label)
            errD_real.backward()
            D_x = output_d_real.mean().item()

            # train with fake
            noise = torch.randn(batch_size, nz, device=device)
            fake = netG(noise)
            label.fill_(fake_label)
            classes_d_fake, output_d_fake = netD(fake.detach())

#                     print("ERR_D_FAKE:",classes_d_fake)
            errD_fake = criterion(classes_d_fake, label)
            errD_fake.backward()
            D_G_z1 = output_d_fake.mean().item()
            errD = errD_real + errD_fake
            optimizerD.step()

            ############################
            # (2) Update G network: minimize Lm(D(G(z)), T=1)
            ###########################
            netG.zero_grad()
            label.fill_(real_label)
            netD.eval()

            classes_g, output_g = netD(fake)
#                     print("ERRG:",classes_g)
            errG = criterion(classes_g, label)
            errG.backward()
            D_G_z2 = output_g.mean().item()
            optimizerG.step()

            print('[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                  % (j + 1, len(train_loader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

            D_losses.append(errD.item())
            G_losses.append(errG.item())

        print("-----THE [{}/{}] epoch end-----".format(epoch + 1, n_epochs))
    
    netD_list.append(netD)
    netG_list.append(netG)
    D_losses_list.append(D_losses)
    G_losses_list.append(G_losses)

In [6]:
input_data = os.listdir(INPUT_PATH)
test_set_list = []

for radiologist_num in range(4): #遍历patient文件夹——study指代每一个study文件夹       
    npy_file_path = os.path.join(INPUT_PATH, "Radiologist_" + str(radiologist_num + 1))
    npy_files = os.listdir(npy_file_path)
    npy_list = []
    for i in npy_files:
        npy_path = os.path.join(npy_file_path, i)
        single_npy = np.load(npy_path)
        single_fliplr_npy = np.fliplr(single_npy)
        single_npy = (single_npy - 127.5) / 127.5
        single_fliplr_npy = (single_fliplr_npy - 127.5) / 127.5
        npy_list.append(single_fliplr_npy)
        npy_list.append(single_npy)

#         print(npy_file_path, np.array(npy_list).shape)
    random.shuffle(npy_list)
    npy_chunks = chunks(npy_list, 5)

    print("THIS IS the data from {}. AND NOW the training STARTS:".format(npy_file_path))
    single_training_set, single_test_set = five_folder(npy_chunks, 4)
    test_set_list.append(single_test_set)
    
    train_loader = torch.utils.data.DataLoader(MyDataset(single_training_set), batch_size=BATCH_SIZE, shuffle=True)
    train(train_loader,single_test_set)


THIS IS the data from ./output/nodule_npy/malignancy/Radiologist_1. AND NOW the training STARTS:
-----THE [1/100] epoch start-----
[1/32] Loss_D: 0.8099 Loss_G: 0.8055 D(x): -0.0000 D(G(z)): -0.0000 / -0.0000
[2/32] Loss_D: 0.7907 Loss_G: 0.7972 D(x): -0.0002 D(G(z)): -0.0000 / -0.0000
[3/32] Loss_D: 0.7769 Loss_G: 0.7901 D(x): -0.0001 D(G(z)): -0.0000 / 0.0000
[4/32] Loss_D: 0.7622 Loss_G: 0.7869 D(x): -0.0001 D(G(z)): 0.0000 / 0.0001
[5/32] Loss_D: 0.7591 Loss_G: 0.7849 D(x): -0.0000 D(G(z)): 0.0000 / 0.0000
[6/32] Loss_D: 0.7452 Loss_G: 0.7859 D(x): -0.0001 D(G(z)): -0.0000 / -0.0000
[7/32] Loss_D: 0.7374 Loss_G: 0.7875 D(x): -0.0002 D(G(z)): -0.0000 / -0.0000
[8/32] Loss_D: 0.7221 Loss_G: 0.7829 D(x): -0.0002 D(G(z)): -0.0000 / -0.0001
[9/32] Loss_D: 0.7232 Loss_G: 0.7773 D(x): -0.0003 D(G(z)): -0.0001 / -0.0001
[10/32] Loss_D: 0.7209 Loss_G: 0.7725 D(x): -0.0002 D(G(z)): -0.0001 / -0.0001
[11/32] Loss_D: 0.7023 Loss_G: 0.7673 D(x): -0.0004 D(G(z)): -0.0001 / -0.0001
[12/32] Loss_D

[5/32] Loss_D: 0.5451 Loss_G: 0.3012 D(x): -0.0038 D(G(z)): -0.0078 / -0.0078
[6/32] Loss_D: 0.5026 Loss_G: 0.3063 D(x): -0.0044 D(G(z)): -0.0079 / -0.0079
[7/32] Loss_D: 0.5714 Loss_G: 0.3060 D(x): -0.0035 D(G(z)): -0.0082 / -0.0080
[8/32] Loss_D: 0.5587 Loss_G: 0.3184 D(x): -0.0041 D(G(z)): -0.0080 / -0.0078
[9/32] Loss_D: 0.4844 Loss_G: 0.2984 D(x): -0.0050 D(G(z)): -0.0080 / -0.0082
[10/32] Loss_D: 0.5360 Loss_G: 0.2952 D(x): -0.0041 D(G(z)): -0.0081 / -0.0080
[11/32] Loss_D: 0.5052 Loss_G: 0.3068 D(x): -0.0047 D(G(z)): -0.0080 / -0.0081
[12/32] Loss_D: 0.5044 Loss_G: 0.3001 D(x): -0.0048 D(G(z)): -0.0079 / -0.0083
[13/32] Loss_D: 0.5881 Loss_G: 0.2984 D(x): -0.0035 D(G(z)): -0.0083 / -0.0082
[14/32] Loss_D: 0.5670 Loss_G: 0.3063 D(x): -0.0039 D(G(z)): -0.0084 / -0.0081
[15/32] Loss_D: 0.5119 Loss_G: 0.3020 D(x): -0.0047 D(G(z)): -0.0083 / -0.0085
[16/32] Loss_D: 0.5492 Loss_G: 0.3012 D(x): -0.0042 D(G(z)): -0.0084 / -0.0085
[17/32] Loss_D: 0.5802 Loss_G: 0.3123 D(x): -0.0035 D(G(z

[11/32] Loss_D: 0.4915 Loss_G: 0.2844 D(x): -0.0059 D(G(z)): -0.0099 / -0.0100
[12/32] Loss_D: 0.5702 Loss_G: 0.2934 D(x): -0.0042 D(G(z)): -0.0099 / -0.0097
[13/32] Loss_D: 0.5191 Loss_G: 0.2949 D(x): -0.0055 D(G(z)): -0.0098 / -0.0096
[14/32] Loss_D: 0.5171 Loss_G: 0.2886 D(x): -0.0055 D(G(z)): -0.0097 / -0.0099
[15/32] Loss_D: 0.4576 Loss_G: 0.2881 D(x): -0.0064 D(G(z)): -0.0099 / -0.0099
[16/32] Loss_D: 0.5593 Loss_G: 0.2998 D(x): -0.0043 D(G(z)): -0.0098 / -0.0096
[17/32] Loss_D: 0.5246 Loss_G: 0.2960 D(x): -0.0051 D(G(z)): -0.0096 / -0.0096
[18/32] Loss_D: 0.5295 Loss_G: 0.2931 D(x): -0.0052 D(G(z)): -0.0096 / -0.0093
[19/32] Loss_D: 0.5163 Loss_G: 0.2826 D(x): -0.0055 D(G(z)): -0.0092 / -0.0094
[20/32] Loss_D: 0.5286 Loss_G: 0.2802 D(x): -0.0052 D(G(z)): -0.0095 / -0.0097
[21/32] Loss_D: 0.5333 Loss_G: 0.2823 D(x): -0.0052 D(G(z)): -0.0097 / -0.0094
[22/32] Loss_D: 0.5062 Loss_G: 0.2836 D(x): -0.0055 D(G(z)): -0.0095 / -0.0094
[23/32] Loss_D: 0.4864 Loss_G: 0.2826 D(x): -0.0061 

[17/32] Loss_D: 0.5402 Loss_G: 0.3370 D(x): -0.0052 D(G(z)): -0.0084 / -0.0083
[18/32] Loss_D: 0.4875 Loss_G: 0.3296 D(x): -0.0064 D(G(z)): -0.0089 / -0.0088
[19/32] Loss_D: 0.3904 Loss_G: 0.3138 D(x): -0.0087 D(G(z)): -0.0099 / -0.0094
[20/32] Loss_D: 0.3906 Loss_G: 0.2362 D(x): -0.0089 D(G(z)): -0.0115 / -0.0118
[21/32] Loss_D: 0.6065 Loss_G: 0.2245 D(x): -0.0047 D(G(z)): -0.0129 / -0.0124
[22/32] Loss_D: 0.4313 Loss_G: 0.1857 D(x): -0.0087 D(G(z)): -0.0142 / -0.0140
[23/32] Loss_D: 0.5180 Loss_G: 0.2003 D(x): -0.0064 D(G(z)): -0.0141 / -0.0136
[24/32] Loss_D: 0.4369 Loss_G: 0.1927 D(x): -0.0085 D(G(z)): -0.0138 / -0.0139
[25/32] Loss_D: 0.4797 Loss_G: 0.1912 D(x): -0.0076 D(G(z)): -0.0142 / -0.0139
[26/32] Loss_D: 0.4982 Loss_G: 0.1932 D(x): -0.0070 D(G(z)): -0.0140 / -0.0138
[27/32] Loss_D: 0.5155 Loss_G: 0.1995 D(x): -0.0067 D(G(z)): -0.0139 / -0.0136
[28/32] Loss_D: 0.5225 Loss_G: 0.2097 D(x): -0.0065 D(G(z)): -0.0137 / -0.0134
[29/32] Loss_D: 0.5146 Loss_G: 0.2210 D(x): -0.0067 

[23/32] Loss_D: 0.5123 Loss_G: 0.2820 D(x): -0.0062 D(G(z)): -0.0111 / -0.0108
[24/32] Loss_D: 0.4193 Loss_G: 0.2672 D(x): -0.0082 D(G(z)): -0.0106 / -0.0109
[25/32] Loss_D: 0.4378 Loss_G: 0.2575 D(x): -0.0076 D(G(z)): -0.0109 / -0.0113
[26/32] Loss_D: 0.4474 Loss_G: 0.2601 D(x): -0.0076 D(G(z)): -0.0112 / -0.0112
[27/32] Loss_D: 0.5165 Loss_G: 0.2675 D(x): -0.0061 D(G(z)): -0.0113 / -0.0109
[28/32] Loss_D: 0.4540 Loss_G: 0.2685 D(x): -0.0072 D(G(z)): -0.0111 / -0.0108
[29/32] Loss_D: 0.4252 Loss_G: 0.2676 D(x): -0.0080 D(G(z)): -0.0107 / -0.0106
[30/32] Loss_D: 0.4900 Loss_G: 0.2738 D(x): -0.0065 D(G(z)): -0.0106 / -0.0104
[31/32] Loss_D: 0.4695 Loss_G: 0.2727 D(x): -0.0070 D(G(z)): -0.0105 / -0.0102
[32/32] Loss_D: 0.4644 Loss_G: 0.2821 D(x): -0.0073 D(G(z)): -0.0103 / -0.0100
-----THE [13/100] epoch end-----
-----THE [14/100] epoch start-----
[1/32] Loss_D: 0.4899 Loss_G: 0.2824 D(x): -0.0063 D(G(z)): -0.0099 / -0.0097
[2/32] Loss_D: 0.4816 Loss_G: 0.2723 D(x): -0.0068 D(G(z)): -0.0

...

[2/32] Loss_D: 0.3803 Loss_G: 0.1672 D(x): -0.0114 D(G(z)): -0.0153 / -0.0152
[3/32] Loss_D: 0.4483 Loss_G: 0.1728 D(x): -0.0093 D(G(z)): -0.0155 / -0.0154
[4/32] Loss_D: 0.4087 Loss_G: 0.1844 D(x): -0.0097 D(G(z)): -0.0152 / -0.0152
[5/32] Loss_D: 0.4105 Loss_G: 0.1817 D(x): -0.0102 D(G(z)): -0.0153 / -0.0151
[6/32] Loss_D: 0.4712 Loss_G: 0.1818 D(x): -0.0087 D(G(z)): -0.0153 / -0.0149
[7/32] Loss_D: 0.4221 Loss_G: 0.1753 D(x): -0.0100 D(G(z)): -0.0148 / -0.0149
[8/32] Loss_D: 0.4123 Loss_G: 0.1760 D(x): -0.0102 D(G(z)): -0.0150 / -0.0152
[9/32] Loss_D: 0.3755 Loss_G: 0.1737 D(x): -0.0107 D(G(z)): -0.0151 / -0.0154
[10/32] Loss_D: 0.4145 Loss_G: 0.1704 D(x): -0.0105 D(G(z)): -0.0152 / -0.0148
[11/32] Loss_D: 0.3631 Loss_G: 0.1657 D(x): -0.0115 D(G(z)): -0.0154 / -0.0155
[12/32] Loss_D: 0.4369 Loss_G: 0.1756 D(x): -0.0091 D(G(z)): -0.0151 / -0.0151
[13/32] Loss_D: 0.4492 Loss_G: 0.1791 D(x): -0.0096 D(G(z)): -0.0157 / -0.0154
[14/32] Loss_D: 0.4436 Loss_G: 0.1863 D(x): -0.0089 D(G(z)):

[8/32] Loss_D: 0.4229 Loss_G: 0.1862 D(x): -0.0093 D(G(z)): -0.0142 / -0.0143
[9/32] Loss_D: 0.4160 Loss_G: 0.1827 D(x): -0.0102 D(G(z)): -0.0149 / -0.0151
[10/32] Loss_D: 0.3486 Loss_G: 0.1696 D(x): -0.0119 D(G(z)): -0.0149 / -0.0150
[11/32] Loss_D: 0.4037 Loss_G: 0.1733 D(x): -0.0101 D(G(z)): -0.0149 / -0.0148
[12/32] Loss_D: 0.3708 Loss_G: 0.1725 D(x): -0.0114 D(G(z)): -0.0147 / -0.0148
[13/32] Loss_D: 0.4170 Loss_G: 0.1689 D(x): -0.0101 D(G(z)): -0.0152 / -0.0151
[14/32] Loss_D: 0.3993 Loss_G: 0.1716 D(x): -0.0102 D(G(z)): -0.0153 / -0.0153
[15/32] Loss_D: 0.3964 Loss_G: 0.1761 D(x): -0.0107 D(G(z)): -0.0155 / -0.0155
[16/32] Loss_D: 0.3951 Loss_G: 0.1759 D(x): -0.0109 D(G(z)): -0.0152 / -0.0151
[17/32] Loss_D: 0.3740 Loss_G: 0.1734 D(x): -0.0114 D(G(z)): -0.0153 / -0.0153
[18/32] Loss_D: 0.4473 Loss_G: 0.1729 D(x): -0.0097 D(G(z)): -0.0155 / -0.0152
[19/32] Loss_D: 0.3561 Loss_G: 0.1730 D(x): -0.0116 D(G(z)): -0.0153 / -0.0153
[20/32] Loss_D: 0.4425 Loss_G: 0.1726 D(x): -0.0093 D(

[14/32] Loss_D: 0.4326 Loss_G: 0.1857 D(x): -0.0094 D(G(z)): -0.0151 / -0.0148
[15/32] Loss_D: 0.3999 Loss_G: 0.1802 D(x): -0.0100 D(G(z)): -0.0150 / -0.0148
[16/32] Loss_D: 0.4229 Loss_G: 0.1834 D(x): -0.0096 D(G(z)): -0.0152 / -0.0150
[17/32] Loss_D: 0.4598 Loss_G: 0.1821 D(x): -0.0087 D(G(z)): -0.0148 / -0.0149
[18/32] Loss_D: 0.3622 Loss_G: 0.1769 D(x): -0.0111 D(G(z)): -0.0148 / -0.0147
[19/32] Loss_D: 0.4081 Loss_G: 0.1798 D(x): -0.0100 D(G(z)): -0.0152 / -0.0152
[20/32] Loss_D: 0.3938 Loss_G: 0.1732 D(x): -0.0105 D(G(z)): -0.0152 / -0.0154
[21/32] Loss_D: 0.4452 Loss_G: 0.1733 D(x): -0.0097 D(G(z)): -0.0151 / -0.0152
[22/32] Loss_D: 0.3670 Loss_G: 0.1725 D(x): -0.0113 D(G(z)): -0.0154 / -0.0150
[23/32] Loss_D: 0.4072 Loss_G: 0.1709 D(x): -0.0101 D(G(z)): -0.0149 / -0.0153
[24/32] Loss_D: 0.4186 Loss_G: 0.1803 D(x): -0.0098 D(G(z)): -0.0150 / -0.0146
[25/32] Loss_D: 0.3924 Loss_G: 0.1732 D(x): -0.0109 D(G(z)): -0.0147 / -0.0148
[26/32] Loss_D: 0.4096 Loss_G: 0.1759 D(x): -0.0101 

[20/32] Loss_D: 0.4131 Loss_G: 0.1749 D(x): -0.0101 D(G(z)): -0.0151 / -0.0152
[21/32] Loss_D: 0.4265 Loss_G: 0.1857 D(x): -0.0095 D(G(z)): -0.0150 / -0.0147
[22/32] Loss_D: 0.4301 Loss_G: 0.1802 D(x): -0.0093 D(G(z)): -0.0149 / -0.0145
[23/32] Loss_D: 0.4893 Loss_G: 0.1835 D(x): -0.0083 D(G(z)): -0.0148 / -0.0147
[24/32] Loss_D: 0.4026 Loss_G: 0.1724 D(x): -0.0099 D(G(z)): -0.0147 / -0.0148
[25/32] Loss_D: 0.4723 Loss_G: 0.1753 D(x): -0.0086 D(G(z)): -0.0148 / -0.0149
[26/32] Loss_D: 0.4569 Loss_G: 0.1740 D(x): -0.0088 D(G(z)): -0.0148 / -0.0145
[27/32] Loss_D: 0.4750 Loss_G: 0.1776 D(x): -0.0082 D(G(z)): -0.0145 / -0.0140
[28/32] Loss_D: 0.4243 Loss_G: 0.1791 D(x): -0.0094 D(G(z)): -0.0142 / -0.0145
[29/32] Loss_D: 0.4020 Loss_G: 0.1774 D(x): -0.0099 D(G(z)): -0.0147 / -0.0147
[30/32] Loss_D: 0.4122 Loss_G: 0.1794 D(x): -0.0099 D(G(z)): -0.0146 / -0.0146
[31/32] Loss_D: 0.3358 Loss_G: 0.1722 D(x): -0.0122 D(G(z)): -0.0144 / -0.0149
[32/32] Loss_D: 0.2801 Loss_G: 0.1871 D(x): -0.0144 

KeyboardInterrupt: 